# Helper functions

The function `convertBasisE10(vE, E, N)` converts a number expressed in base-`E` (and represented as the vector `vE`) into its decimal representation `p` (as an integer).

*Convention:* `vE(1)` represents the idiosyncratic  state in the present period and `vE(N)` represents the past idosyncratic state N-1 periods

In [1]:
function convertBasisE10(vE::Vector{I}, E::I)::I where {I<:Integer}
# converts the a vector vE into an integer p 

    @assert (minimum(vE)≥1)&&(maximum(vE)≤E)
    
    N = length(vE)    
    p = one(I) #otherwise p lieas between 0 and (E+1)^N - 1
    for k ∈ eachindex(vE)
        p += (vE[k]-1)*E^(N-k)
    end
    return p
end

convertBasisE10 (generic function with 1 method)

The function `convertBasis10E(p, E, N)` converts a number expressed in base-10 (and represented as the integer p) into its base-`E` representation `vE` (as a vector of length `N`). This is the inverse of `convertBasisE10`.

*Convention:* `vE(1)` represents the idiosyncratic  state in the present period and `vE(N)` represents the past idosyncratic state N-1 periods

In [4]:
function  convertBasis10E(p::I, E::I, N::I)::Vector{I} where{I<:Integer}

    vE = zeros(I, N)    
    @assert (p≥1)&&(p≤E^N)
    ptemp = p-1;
    for k ∈ eachindex(vE)
        ptemp, r = divrem(ptemp,E)
        vE[N-k+1] = 1+r        
    end
    return vE
end

convertBasis10E (generic function with 1 method)

The function `historySizes(N::I, Πy::Matrix{T};
    maxiter::I=1000000, tol::T=1e-16)` computes the size of (positive-size) truncated histories. It takes as input:
* `N`: the truncation length;
* `Πy`: the transition matrix of the productivity process;
* `maxiter` and `tol`: controls for the convergence of the function `powm!` to compute the eigenvector of `Πy`.
    
The function returns a t-uple with three elements:
* a list of indices `ind_h` corresponding to positive-size histories;
* a list of values `S_h` corresponding to the positive sizes of  histories; note that the pair (`ind_h`,`S_h`) allows one to reconstruct the sparse vector of history sizes). 
* the list of prodcutivity levels `y0h[ind_h]` of positive-size histories.

In [ ]:
#Constructs a sparse vector of the size of truncated history
#The indices of the sparse vector (i.e. of non zero size) provides the list of non-empty histories
function historySizes(N::I, Πy::Matrix{T};
        maxiter::I=1000000, tol::T=1e-16) where{I<:Integer, T<:Real}
    ny = size(Πy,1)
    
    _, Sy = powm!(Πy', ones(typeof(Πy[1]), ny),maxiter = maxiter,tol = tol)
    Sy /= sum(Sy)
    Ntot = ny^N #total nb of histories
    Sh   = zeros(typeof(Πy[1]), Ntot) # size of truncated histories
    y0h   = zeros(I, Ntot) # current productivity indices of history
    
    # We loop over all truncated histories
    for h ∈ eachindex(Sh)
        vE =  convertBasis10E(h, ny, N) 
                    # vector of producitvity levels for 
                    # truncated history h
        Sh[h] = Sy[vE[N]] 
                    # distribution according to the 
                    # terminal productity level
        y0h[h] = vE[1]
                    # current prod. level of h
        for j = N-1:-1:1
            Sh[h] *= Πy[vE[j+1],vE[j]] 
            # We move backwards and compute the probability to move 
            # from productivity vE[j] to productivity vE[j+1]
        end  
    end
    ind_h, S_h = findnz(sparsevec(Sh))
    return ind_h, S_h, y0h[ind_h]
end



The function `historyDist(ny::I, N::I, stationaryDist::Matrix{T}, 
    transitMat::SparseMatrixCSC{T,I})` computes the distribution of (positive-size) truncated histories. It takes as input:
* `ny`: the nb of productivity states;
* `N`: the truncation length;
* `stationaryDist`: the stationary distribution over truncated histories; 
* `transitMat`: the transition matrix over the product grid asset $\times$ productivity. 

The function returns a matrix $((na \times ny), ny^N)$ containing the distribution of truncated histories over the product grid asset $\times$ productivity. More precisely, if `statDist_h = historyDist(ny, N, stationaryDist, transitMat)`, then  `statDist_h[:,h]` is the distribution of truncated history `h` over the product grid asset $\times$ productivity.

In [ ]:

function historyDist(ny::I, N::I, stationaryDist::Matrix{T}, 
            transitMat::SparseMatrixCSC{T,I})::Matrix{T} where{I<:Integer, T<:Real}
    
    na = div(size(transitMat,1),ny)
    Nh = ny^N
    statDist_h = zeros(T, length(stationaryDist), Nh)
    temp_v = zeros(T, na)
    for i ∈ 1:Nh
        vE     = convertBasis10E(i, ny, N)
        shift0 = (vE[N]-1)*na
        statDist_h[1+shift0:na+shift0,i] =  stationaryDist[:,vE[N]]
        for j=N-1:-1:1                
            shift1 = (vE[j+1]-1)*na
            shift0 = (vE[j]-1)*na
            temp_v .= statDist_h[1+shift1:na+shift1,i]
            statDist_h[:,i] .= zero(T)
            statDist_h[1+shift0:na+shift0,i] .= (
                transitMat[1+shift0:na+shift0,1+shift1:na+shift1] * temp_v)
        end
    end
    return statDist_h
end


The function `historyTrans(N::I,ind_h::Vector{I},Πy::Matrix{T})` computes the transition matrix for truncated histories. It takes as input:
* `N`: the truncation length;
* `ind_h`: the index of non-zero size truncated histories;
* `Πy`: the productivity transition matrix.
        
The function returns a matrix `((na` $\times$ `ny), ny^N)` containing the distribution of truncated histories over the product grid asset $\times$ productivity. More precisely, if `statDist_h = historyDist(ny, N, stationaryDist, transitMat)`, then  `statDist_h[:,h]` is the distribution of truncated history `h` over the product grid asset $\times$ productivity.

In [ ]:

function historyTrans(N::I,ind_h::Vector{I},Πy::Matrix{T}) where{I<:Integer,T<:Real}
    ny = length(Πy[:,1])
    Ntot = length(ind_h)
    Πh = spzeros(T,Ntot,Ntot)
    for (i,h) ∈ enumerate(ind_h)
        vE_h = convertBasis10E(h, ny, N)
        for (j,ht) ∈ enumerate(ind_h)
            vE_ht = convertBasis10E(ht, ny, N)
            if all(vE_ht[2:end] .== vE_h[1:end-1])
                Πh[i,j] = Πy[vE_h[1],vE_ht[1]]
            end
        end
    end
#     @show "here"
    return Πh
end


The function `credit_constrained_h(shareCC::T, S_h::Vector{T}, x_h::Vector{T};method::String="first larger")` identifies credit-constraint histories. It takes as input:
* `shareCC`: the target share of credit-constrained agents;
* `S_h`: the distribution of truncated histories;
* `x_h`: a vector of the same size as `S_h`;
* `method`: a string caharacterizing the selction mechanism.
        
The function returns a pair:
* an integer `i_cc` indicating the number of credit-constrained truncated histories;
* a vector of indices of credit-constrained truncated histories.

The credit-constrained histories are computed such that based the first `i_cc` truncated histories (ordered along `x_h`) have a size similar to the target `shareCC`. The meaning of "similar" is specified by the method: either the first larger or the closest.

In [ ]:
#defined below


# The Refined Truncation function

## The main function

In [ ]:
function truncate_polfun(x_ij::Matrix{T}, statDist_h::Matrix{T}; f=identity) where{T<:Real}
    na,ny  = size(x_ij)
    n,Ntot = size(statDist_h)
    @assert n==na*ny
    
#     foo = x -> (isinf(f(x))) ? 1e200 : f(x)
#     foo = identity
    
#     @show sum(reshape(statDist_h,na,ny,Ntot),dims=1)
#     @show foo.(sum(reshape(statDist_h,na,ny,Ntot).*repeat(g,1,1,Ntot),dims=1))
#     @show sum(sum(reshape(statDist_h,na,ny,Ntot),dims=1),dims=2)[:]
#     @show sum(reshape(statDist_h,na,ny,Ntot).*repeat(g,1,1,Ntot),dims=1)
    
#     @show size(reshape(statDist_h,na,ny,Ntot))
#     @show size(repeat(sum(reshape(statDist_h,na,ny,Ntot),dims=1),10))
    p_ijk = reshape(statDist_h,na,ny,Ntot) 
    toR = zeros(T, Ntot)
    for k in 1:Ntot
        p_j = sum(p_ijk[:,:,k],dims=1)
        x_j = sum(p_ijk[:,:,k].*x_ij,dims=1)
        f_x_j = zeros(T, size(x_j))
        f_x_j[x_j.>zero(T)] = f.(x_j[x_j.>zero(T)]./(sum(p_ijk[:,:,k],dims=1)[x_j.>zero(T)]))
#         if any(isnan.(f_x_j))
#             @show sum(p_ijk[:,:,k].*x_ij,dims=1), sum(p_ijk[:,:,k],dims=1)
#         end
#         @show size((sum(p_j.*f_x_j,dims=2)./sum(p_j,dims=2))[1,1])
        toR[k] = (sum(p_j.*f_x_j,dims=2)./sum(p_j,dims=2))[1,1]
#         if isnan(toR[k])
#             @show sum(p_j.*f_x_j,dims=2), sum(p_j,dims=2)
#         end
    end
    return toR
end


In [ ]:
function refinedcredit_constrained_h(shareCC::T, S_h::Vector{T}, x_h::Vector{T}; 
    method::String="closest",rev::Bool=true, a_min::T=nothing) where{T<:Real}
ind_x_indiv = sortperm(x_h,rev=rev)
S_h_sorted = S_h[ind_x_indiv]

#  if !isnothing(a_min)
#     ind_x_indiv_ = findall(<=(2*a_min), x_h)
#     i_c_ = length(ind_x_indiv_)
# end     
        
if method=="closest"
    i_cc = argmin(abs.(cumsum(S_h_sorted) .- shareCC)) #closest distance
else #(method=="first larger")
    #i_cc = findlast(x->x<=0.0, cumsum(S_h_sorted) .- shareCC)#at least as large
    #i_cc = findlast(x->x<0.0, cumsum(S_h_sorted) .- shareCC)#at least as large
    i_cc = findfirst(x->x>0.0, cumsum(S_h_sorted) .- shareCC)#at least as large
end
i_c = isnothing(i_cc) ? 1 : i_cc

return i_c, ind_x_indiv[1:i_c]
# if i_c_ > i_c
#     return i_c_, ind_x_indiv_
# else
#     return i_c, ind_x_indiv[1:i_c]
# end
end;


function refinedHistoryTrans(hs::Vector{Vector{I}},Πy::Matrix{T}) where{I<:Integer,T<:Real}
    ny = length(Πy[:,1])
    Ntot = length(hs)
    Πh = spzeros(T,Ntot,Ntot)
    nh, nht = zero(I), zero(I)
    for (i,h) ∈ enumerate(hs)
        nh = length(h)
        for (j,ht) ∈ enumerate(hs)
            nht = length(ht)
            if (nh≥nht)&&all(ht[2:end] .== h[1:nht-1])
                Πh[i,j] = Πy[h[1],ht[1]]
            elseif (nht>nh) && all(ht[2:(nh+1)] .== h)
                Πh[i,j] = Πy[h[1],ht[1]]
            end
        end
    end
    return Πh
end;

function refinedHistoryDist(ny::I,hs::Vector{Vector{I}},stationaryDist::Matrix{T}, 
    transitMat::SparseMatrixCSC{T,I})::Matrix{T} where{I<:Integer, T<:Real}

na = div(size(transitMat,1),ny) # size of asset grid 
Ntot = length(hs) # nb of truncated histories
#     @show stationaryDist
#     @show transitMat
# initialization of the stat distribution: matrix of size (na*ny, Ntot)
statDist_h = zeros(T, length(stationaryDist), Ntot)
temp_v = zeros(T, na)
nh = zero(I)
# Loop over all truncated histories
for (ih,h) ∈ enumerate(hs)
shift0 = (h[end]-1)*na # shift for the index of vE[N] in the product grid
statDist_h[1+shift0:na+shift0,ih] =  stationaryDist[:,h[end]]
nh = length(h)
for j=nh-1:-1:1                
    shift1 = (h[j+1]-1)*na
    shift0 = (h[j]-1)*na
    temp_v .= statDist_h[1+shift1:na+shift1,ih]
    statDist_h[:,ih] .= zero(T)
    statDist_h[1+shift0:na+shift0,ih] .= (
        transitMat[1+shift0:na+shift0,1+shift1:na+shift1] * temp_v)
end
end
return statDist_h
end;



function refinedHistorySizes(hs::Vector{Vector{I}}, Πy::Matrix{T};
    maxiter::I=1000000, tol::T=1e-16) where{I<:Integer, T<:Real}

ny = size(Πy,1) # nb of producitivty states

_,Sy = powm!(Πy', ones(typeof(Πy[1]), ny),maxiter = maxiter,tol = tol)
Sy  /= sum(Sy)  # distribution over producitivity levels
Ntot = length(hs)     # total nb of truncated histories
Sh   = zeros(typeof(Πy[1]), Ntot) 
                # initialization of size of truncated histories
y0h   = zeros(I, Ntot) 
                # initialization of current productivity indices of history

# We loop over all truncated histories
nh = zero(I)
for (ih,h) ∈ enumerate(hs)
    nh = length(h)
    Sh[ih] = Sy[h[nh]] 
                # distribution according to the 
                # terminal productity level
    y0h[ih] = h[1]
                # current prod. level of h
    for j = nh-1:-1:1
        Sh[ih] *= Πy[h[j+1],h[j]] 
        # We move backwards and compute the probability to move 
        # from productivity vE[j] to productivity vE[j+1]
    end  
end
ind_h, S_h = findnz(sparsevec(Sh))
return ind_h, S_h, y0h[ind_h]
end;


function generateHistories(N::I,ny::I)::Vector{Vector{I}} where I<:Int
    @assert N≥0 && ny≥0
    if N==0
        return [[]]
    elseif N==1
        return map(n->[n],1:ny)
    else
        return mapreduce(x -> map(n->push!(copy(x),n),1:ny),vcat,generateHistories(N-1,ny))
    end
end

function refine_one_hist(x::Vector{I},ny::I)::Vector{Vector{I}} where I<:Int
    if all(x.==x[1])
        return map(n->push!(copy(x),n),1:ny)
    else
        return [copy(x)]
    end
end

function refine_hist(Nr::I,hs::Vector{Vector{I}},ny::I)::Vector{Vector{I}} where I<:Int
    if Nr<=0
        return hs
    else
        refine_hist(Nr-1,mapreduce(x -> refine_one_hist(x,ny),vcat,hs),ny)
    end
end



# Vidio is the vector of length ny, which gives the length of the refinment by idiosyncratic state
function refinedbuild_histories(N::I,Vidio::Vector{I},ny::I)::Vector{Vector{I}} where I<:Int
    
    hs  = generateHistories(N,ny)
    # first one
    Vid = (repeat([1],N)) #  refine hte [1,..1] history
    res_i=convertBasisE10(Vid,ny)
    hfin = refine_hist(Vidio[1]-N,[hs[res_i]],ny)
    res_n = res_i  # I store the index of the last refined history
    for i=2:ny # refine other histories
        Vid = (repeat([i],N))
        res_i=convertBasisE10(Vid,ny)
        hfin =  vcat(hfin,hs[res_n+1:res_i-1])
        hr = refine_hist(Vidio[i]-N,[hs[res_i]],ny)
        res_n = res_i
        hfin =  vcat(hfin,hr)
    end
    return hfin
end


In [ ]:

function refinedProject_plan(N::Integer,Vidio::Vector{Int64}, # length of the truncation
    solution::AiyagariSolution, 
    params::Params;maxiter=1000000,tol=1e-16,new_ξs=true)

#@unpack β,α,δ,γ,u,u′,u′′,na,a_min,aGrid,ny,ys,Πy = params

#@unpack ga,gc,R,w,A,K,L,transitMat,stationaryDist,residEuler = solution
@unpack β,α,δ,tk,tl,u,u′,u′′,v,v′,l_supply,na,a_min,aGrid,ny,ys,Πy = params
@unpack ga,gc,gl,R,w, A,K,L,transitMat,stationaryDist,residEuler = solution

# Identifying positive-size truncated histories
hs = refinedbuild_histories(N,Vidio,ny)
(ind_h, S_h, ind_y0_h)= refinedHistorySizes(hs,Πy)
#@show length(hs)
hs =hs[ind_h] # Important : I redefine the histories here to consider the relevant one
y0_h = [ys[i] for i in ind_y0_h] # or y0_h =ys[ind_h]


T = typeof(Πy[1,1])


# Distribution of truncated histories
#statDist_h = refinedHistoryDist(ny, hs, stationaryDist, transitMat)[:,ind_h]
statDist_h = refinedHistoryDist(ny, hs, stationaryDist, transitMat)
Ntot = length(hs)#nb of history with positive size
Π_h = refinedHistoryTrans(hs,Πy)

#Compute truncated allocations
c_h       = truncate_polfun(gc, statDist_h)
#     @show size(gc), size(reshape(statDist_h,na,ny,Ntot))
#     @show gc
#     @show reshape(statDist_h,na,ny,Ntot)

#     @show c_h .- sum(statDist_h .* repeat(gc[:],1,Ntot), dims=1)[:]./S_h 
#     return
a_beg_h   = truncate_polfun(repeat(aGrid[:],ny,1), statDist_h)
a_end_h   = truncate_polfun(ga, statDist_h)

v_h       = truncate_polfun(gl, statDist_h, f=v)
v′_h      = truncate_polfun(gl, statDist_h, f=v′)
l_h       = truncate_polfun(gl, statDist_h) # f= identity
ly_h      = truncate_polfun((repeat(ys',na,1).*gl), statDist_h) # f= identity


#v_h       = sum(statDist_h .* repeat(v.(gl[:]),1,Ntot), dims=1)[:]./S_h
#v′_h      = sum(statDist_h .* repeat(v′.(gl[:]),1,Ntot), dims=1)[:]./S_h
#ly_h  = sum(statDist_h .* repeat(((repeat(ys',na,1).*gl)[:]),1,Ntot), dims=1)[:]./S_h



resid_E_h = truncate_polfun(residEuler, statDist_h)

if new_ξs
    u_h       = truncate_polfun(gc, statDist_h, f=u)
    u′_h      = truncate_polfun(gc, statDist_h, f=u′)
    u′′_h     = truncate_polfun(gc, statDist_h, f=u′′)
else
    u_h       = sum(statDist_h .* repeat(u.(gc[:]),1,Ntot), dims=1)[:]./S_h
    u′_h      = sum(statDist_h .* repeat(u′.(gc[:]),1,Ntot), dims=1)[:]./S_h
    u′′_h     = sum(statDist_h .* repeat(u′′.(gc[:]),1,Ntot), dims=1)[:]./S_h
end

#     @show sum(statDist_h .* repeat(gc[:],1,Ntot), dims=1)[:]./S_h
#     @show c_h .- sum(statDist_h .* repeat(gc[:],1,Ntot), dims=1)[:]./S_h   
#     @show l_h .- sum(statDist_h .* repeat(gl[:],1,Ntot), dims=1)[:]./S_h   
#     @show C_h .- sum(statDist_h .* repeat(gC[:],1,Ntot), dims=1)[:]./S_h   
#     @show a_beg_h .- sum(statDist_h .* repeat(aGrid[:],ny,Ntot), dims=1)[:]./S_h
#     @show a_end_h .- sum(statDist_h .* repeat(ga[:],1,Ntot), dims=1)[:]./S_h
# #     @show resid_E_h .- sum(statDist_h .* repeat(residEuler[:],1,Ntot), dims=1)[:]./S_h



# Indentify credit constrained histories
#     @show stationaryDist[1,:], sum(stationaryDist)
#     @show stationaryDist[2,:]
share_cc = sum(stationaryDist[1,:]) #Share of credit constrained agents


#share_cc =0.2974393680642055
#nb_cc_h, ind_cc_h = refinedcredit_constrained_h(share_cc, S_h,resid_E_h,method="closest",rev=true,a_min=params.a_min)
#nb_cc_h, ind_cc_h = refinedcredit_constrained_h(share_cc, S_h,resid_E_h,method="else",rev=true,a_min=params.a_min)
nb_cc_h, ind_cc_h = refinedcredit_constrained_h(share_cc, S_h,resid_E_h,method="closest",rev=true,a_min=params.a_min)



#    nb_cc_h, ind_cc_h = credit_constrained_h(share_cc,S_h,a_end_h,method="closest",rev=false,a_min=a_min)


# The truncated allocation  structure
allocation_proj = Allocation_proj(
S_h=S_h,
Π_h=Π_h,
y0_h=y0_h,
a_beg_h=a_beg_h,
a_end_h=a_end_h,
c_h=c_h,
l_h=l_h,
ly_h=ly_h,
u_h=u_h,
u′_h=u′_h,
u′′_h=u′′_h,
v_h=v_h,
v′_h=v′_h,
resid_E_h=resid_E_h,
nb_cc_h=nb_cc_h,
ind_cc_h=ind_cc_h)

# The ξs allocation structure
ξu0 = u_h./u.(c_h)
ξu1 = u′_h./u′.(c_h)
ξu2 = u′′_h./u′′.(c_h)

ξuE = ((I-β*R*Π_h)\resid_E_h)./u′.(c_h)
ξy  = ly_h./(y0_h.*l_h)
ξv0 = v_h./v.(l_h)
ξv1 = w*y0_h.*u′_h./(v′.(l_h)) #(equivalent to τ*w*ξy*.(y0_h.*l_h).^τ?*ξu1.*u′.(c_h)./l_h)

ξs = ξs_struct(
    ξu0=ξu0,
    ξu1=ξu1,
    ξu2=ξu2,
    ξuE=ξuE,
    ξy=ξy,
    ξv0=ξv0,
    ξv1=ξv1)

return Projection(
N                = N, # here common histories
Ntot             = Ntot,
ind_h            = ind_h,# here useless (i.e. from 1 to Ntot)
allocation_proj = allocation_proj,
ξs               = ξs)

end;



In [ ]:

function test(N::Integer, # length of the truncation
    solution::AiyagariSolution, 
    params::Params;maxiter=1000000,tol=1e-16)
@unpack β,α,δ,tk,tl,u,u′,u′′,v,v′,l_supply,na,a_min,aGrid,ny,ys,Πy = params
@unpack ga,gc,gl,R,w, A,K,L,transitMat,stationaryDist,residEuler = solution
(ind_h, S_h, ind_y0_h) = historySizes(N, Πy)
y0_h = [ys[i] for i in ind_y0_h]
T = typeof(Πy[1,1])
statDist_h = historyDist(ny, N, stationaryDist, transitMat)[:,ind_h]
Ntot = length(ind_h)#nb of history with positive size
Π_h = historyTrans(N,ind_h,Πy)

#Compute allocations
c_h       = sum(statDist_h .* repeat(gc[:],1,Ntot), dims=1)[:]./S_h   
l_h       = sum(statDist_h .* repeat(gl[:],1,Ntot), dims=1)[:]./S_h
a_beg_h   = sum(statDist_h .* repeat(aGrid[:],ny,Ntot), dims=1)[:]./S_h
a_end_h   = sum(statDist_h .* repeat(ga[:],1,Ntot), dims=1)[:]./S_h
u_h       = sum(statDist_h .* repeat(u.(gc[:]),1,Ntot), dims=1)[:]./S_h
u′_h      = sum(statDist_h .* repeat(u′.(gc[:]),1,Ntot), dims=1)[:]./S_h
u′′_h     = sum(statDist_h .* repeat(u′′.(gc[:]),1,Ntot), dims=1)[:]./S_h
v_h       = sum(statDist_h .* repeat(v.(gl[:]),1,Ntot), dims=1)[:]./S_h
v′_h      = sum(statDist_h .* repeat(v′.(gl[:]),1,Ntot), dims=1)[:]./S_h
# ly_h  = sum(statDist_h .* repeat(((repeat(ys',na,1).*gl)[:]).^τ,1,Ntot), dims=1)[:]./S_h
ly_h  = sum(statDist_h .* repeat(((repeat(ys',na,1).*gl)[:]),1,Ntot), dims=1)[:]./S_h
resid_E_h = sum(statDist_h .* repeat(residEuler[:],1,Ntot), dims=1)[:]./S_h

#Define credit constrained histories
share_cc = sum(stationaryDist[1,:]) #Share of credit constrained agents
nb_cc_h, ind_cc_h = credit_constrained_h(share_cc, S_h, c_h;method="first larger")

allocation_proj = Allocation_proj(
S_h=S_h,
Π_h=Π_h,
y0_h=y0_h,
a_beg_h=a_beg_h,
a_end_h=a_end_h,
c_h=c_h,
l_h=l_h,
ly_h=ly_h,
u_h=u_h,
u′_h=u′_h,
u′′_h=u′′_h,
v_h=v_h,
v′_h=v′_h,
resid_E_h=resid_E_h,
nb_cc_h=nb_cc_h,
ind_cc_h=ind_cc_h)



return resid_E_h

end



